In [11]:
import numpy as np
import pandas as pd
import time
import threading
import sys

In [12]:
from PyQt5.QtCore import QTimer, QTime, Qt

In [ ]:
# gold mine created automatically
import time
import threading
import pandas as pd
import os

# -----------------------------
# INITIAL GAME STATE
# -----------------------------
goldMine = 1                      # Start with 1 goldmine
initial_gold = 280.0
first_mine_cost = 150
gold = initial_gold - first_mine_cost  # Subtract first goldmine cost
multiplier = 5 + (goldMine / 2)
start_time = time.time()
running = True
lock = threading.Lock()

# -----------------------------
# DATA STORAGE
# -----------------------------
gold_history = pd.DataFrame(columns=["seconds", "gold", "goldMine"])
max_duration = 2100  # 35 minutes

# -----------------------------
# GOLD ACCUMULATION FUNCTION
# -----------------------------
def gold_function():
    global gold, multiplier, gold_history, running
    last_update_time = time.time()

    while running:
        time.sleep(1)
        with lock:
            current_time = time.time()
            elapsed_time = current_time - start_time

            if elapsed_time >= max_duration:
                print("\n⏰ 35 minutes have passed. Ending game.")
                running = False
                break

            delta_time = current_time - last_update_time
            last_update_time = current_time

            # Update multiplier and accumulate gold
            multiplier = 5 + (goldMine / 2)
            gold += delta_time * multiplier
            seconds_passed = int(elapsed_time)

            # Log values
            gold_history.loc[len(gold_history)] = [seconds_passed, int(gold), goldMine]

            # Display output
            print("\n--- Game Status ---")
            print(f"Time: {seconds_passed} sec")
            print(f"Gold: {int(gold)}")
            print(f"Multiplier: {multiplier}")
            print(f"Goldmines: {goldMine}")

# -----------------------------
# USER INPUT FUNCTION
# -----------------------------
def get_user_input():
    global gold, goldMine, running
    last_mine_time = start_time  # cooldown timer starts at beginning
    cooldown = 90

    while running:
        user_input = input("\nEnter 'mine' to build a Goldmine (90 sec cooldown), or 'quit' to exit: ").strip().lower()

        if user_input == "mine":
            current_time = time.time()
            with lock:
                if current_time - last_mine_time >= cooldown:
                    cost = 150 + goldMine * 50  # initial mine cost is 150
                    if gold >= cost:
                        goldMine += 1
                        gold -= cost
                        last_mine_time = current_time
                        print(f"✅ Built Goldmine #{goldMine}! (-{cost} gold)")
                    else:
                        print(f"❌ Not enough gold! You need {cost}, but have {int(gold)}.")
                else:
                    remaining = int(cooldown - (current_time - last_mine_time))
                    print(f"⏳ Goldmine is on cooldown. Wait {remaining} more seconds.")
        elif user_input == "quit":
            with lock:
                running = False
                print("💾 Saving game data...")
                break
        else:
            print("❓ Invalid command. Type 'mine' or 'quit'.")

# -----------------------------
# SAVE FUNCTION
# -----------------------------
def save_gold_history():
    global gold_history
    filename = "gold_progress.csv"
    run_id = 1

    if os.path.exists(filename):
        existing = pd.read_csv(filename)

        while f"run_{run_id}_gold" in existing.columns:
            run_id += 1

        new_data = gold_history.copy()
        new_data.rename(columns={
            "gold": f"run_{run_id}_gold",
            "goldMine": f"run_{run_id}_goldMine"
        }, inplace=True)

        merged = pd.merge(existing, new_data, on="seconds", how="outer").sort_values(by="seconds")
    else:
        gold_history.rename(columns={
            "gold": f"run_1_gold",
            "goldMine": f"run_1_goldMine"
        }, inplace=True)
        merged = gold_history

    merged.to_csv(filename, index=False)
    print(f"✅ Game data saved as Run #{run_id} to {filename}")

# -----------------------------
# MAIN THREAD HANDLING
# -----------------------------
try:
    function_thread = threading.Thread(target=gold_function)
    input_thread = threading.Thread(target=get_user_input)

    function_thread.start()
    input_thread.start()

    function_thread.join()
    input_thread.join()
except KeyboardInterrupt:
    with lock:
        running = False
        print("\n🛑 Keyboard interrupt detected. Exiting gracefully...")

# -----------------------------
# FINAL SAVE ON EXIT
# -----------------------------
with lock:
    save_gold_history()
    print("🎮 Game ended.")


In [13]:
#base run
import time
import threading
import pandas as pd
import os
# Global game state
gold = 280.0
goldMine = 0
multiplier = 5.0
start_time = time.time()
running = True
lock = threading.Lock()

# DataFrame to track history
gold_history = pd.DataFrame(columns=["seconds", "gold", "goldMine"])
max_duration = 2100  # 35 minutes in seconds

def gold_function():
    global gold, multiplier, gold_history, running
    last_update_time = time.time()

    while running:
        time.sleep(1)
        with lock:
            current_time = time.time()
            elapsed_time = current_time - start_time

            if elapsed_time >= max_duration:
                print("\n⏰ 35 minutes have passed. Ending game.")
                running = False
                break

            delta_time = current_time - last_update_time
            last_update_time = current_time

            # Update multiplier and gold
            multiplier = 5 + (goldMine / 2)
            gold += delta_time * multiplier
            seconds_passed = int(elapsed_time)

            # Log the values
            gold_history.loc[len(gold_history)] = [seconds_passed, int(gold), goldMine]

            # Output for user
            print("\n--- Game Status ---")
            print(f"Time: {seconds_passed} sec")
            print(f"Gold: {int(gold)}")
            print(f"Multiplier: {multiplier}")
            print(f"Goldmines: {goldMine}")

def get_user_input():
    global gold, goldMine, running
    last_mine_time = 0
    cooldown = 90  # 90 seconds cooldown for goldmines

    while running:
        user_input = input("\nEnter 'mine' to build a Goldmine (90 sec cooldown), or 'quit' to exit: ").strip().lower()

        if user_input == "mine":
            current_time = time.time()
            with lock:
                if current_time - last_mine_time >= cooldown:
                    cost = 150 + goldMine * 50
                    if gold >= cost:
                        goldMine += 1
                        gold -= cost
                        last_mine_time = current_time
                        print(f"✅ Built Goldmine #{goldMine}! (-{cost} gold)")
                    else:
                        print(f"❌ Not enough gold! You need {cost}, but have {int(gold)}.")
                else:
                    remaining = int(cooldown - (current_time - last_mine_time))
                    print(f"⏳ Goldmine is on cooldown. Wait {remaining} more seconds.")
        elif user_input == "quit":
            with lock:
                running = False
                print("💾 Saving game data...")
                break
        else:
            print("❓ Invalid command. Type 'mine' or 'quit'.")

# Start threads
function_thread = threading.Thread(target=gold_function)
input_thread = threading.Thread(target=get_user_input)

function_thread.start()
input_thread.start()

function_thread.join()
input_thread.join()

def save_gold_history():
    global gold_history

    filename = "gold_progress.csv"
    run_id = 1

    # Load existing file if it exists
    if os.path.exists(filename):
        existing = pd.read_csv(filename)

        # Determine next run number by checking column names
        while f"run_{run_id}_gold" in existing.columns:
            run_id += 1

        # Prepare new columns for current run
        new_data = gold_history.copy()
        new_data.rename(columns={
            "gold": f"run_{run_id}_gold",
            "goldMine": f"run_{run_id}_goldMine"
        }, inplace=True)

        # Merge on seconds
        merged = pd.merge(existing, new_data, on="seconds", how="outer").sort_values(by="seconds")
    else:
        # First run: rename columns and use as-is
        gold_history.rename(columns={
            "gold": f"run_{run_id}_gold",
            "goldMine": f"run_{run_id}_goldMine"
        }, inplace=True)
        merged = gold_history

    # Save merged data back
    merged.to_csv(filename, index=False)
    print(f"✅ Game data saved as Run #{run_id} to {filename}")

# Save game data once both threads exit
with lock:
    save_gold_history()
    print("🎮 Game ended.")


--- Game Status ---
Time: 1 sec
Gold: 280
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 2 sec
Gold: 285
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 3 sec
Gold: 290
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 4 sec
Gold: 295
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 5 sec
Gold: 300
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 6 sec
Gold: 305
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 7 sec
Gold: 310
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 8 sec
Gold: 315
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 9 sec
Gold: 320
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 10 sec
Gold: 325
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 11 sec
Gold: 330
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 12 sec
Gold: 335
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 13 sec
Gold: 340
Multiplier: 5.0
Goldmines: 0

--- Game Status ---
Time: 14 sec
Gold: 345
Multiplier: 5.0



Enter 'mine' to build a Goldmine (90 sec cooldown), or 'quit' to exit:  quit


💾 Saving game data...
✅ Game data saved as Run #1 to gold_progress.csv
🎮 Game ended.


In [ ]:
#2nd goldmine created at 3 minutes
import time
import threading
import pandas as pd
import os

# -----------------------------
# INITIAL GAME STATE
# -----------------------------
goldMine = 1                      # Start with 1 goldmine
initial_gold = 280.0
first_mine_cost = 150
gold = initial_gold - first_mine_cost  # Subtract first goldmine cost
multiplier = 5 + (goldMine / 2)
start_time = time.time()
running = True
lock = threading.Lock()

# -----------------------------
# DATA STORAGE
# -----------------------------
gold_history = pd.DataFrame(columns=["seconds", "gold", "goldMine"])
max_duration = 2100  # 35 minutes
auto_mine_added = False  # Flag to ensure only one auto-mine at 3 mins

# -----------------------------
# GOLD ACCUMULATION FUNCTION
# -----------------------------
def gold_function():
    global gold, multiplier, gold_history, running, auto_mine_added, goldMine
    last_update_time = time.time()

    while running:
        time.sleep(1)
        with lock:
            current_time = time.time()
            elapsed_time = current_time - start_time

            if elapsed_time >= max_duration:
                print("\n⏰ 35 minutes have passed. Ending game.")
                running = False
                break

            # Add automatic goldmine after 3 minutes (180 sec)
            if not auto_mine_added and elapsed_time >= 180:
                cost = 150 + goldMine * 50  # Use same cost scaling
                if gold >= cost:
                    gold -= cost
                    goldMine += 1
                    auto_mine_added = True
                    print(f"\n⚒️ Auto-built Goldmine #{goldMine} at 3 minutes! (-{cost} gold)")
                else:
                    print(f"\n⚠️ Auto-build failed: not enough gold for 2nd mine at 3 minutes.")

            delta_time = current_time - last_update_time
            last_update_time = current_time

            # Update multiplier and accumulate gold
            multiplier = 5 + (goldMine / 2)
            gold += delta_time * multiplier
            seconds_passed = int(elapsed_time)

            # Log values
            gold_history.loc[len(gold_history)] = [seconds_passed, int(gold), goldMine]

            # Display output
            print("\n--- Game Status ---")
            print(f"Time: {seconds_passed} sec")
            print(f"Gold: {int(gold)}")
            print(f"Multiplier: {multiplier}")
            print(f"Goldmines: {goldMine}")

# -----------------------------
# USER INPUT FUNCTION
# -----------------------------
def get_user_input():
    global gold, goldMine, running
    last_mine_time = start_time  # cooldown timer starts at beginning
    cooldown = 90

    while running:
        user_input = input("\nEnter 'mine' to build a Goldmine (90 sec cooldown), or 'quit' to exit: ").strip().lower()

        if user_input == "mine":
            current_time = time.time()
            with lock:
                if current_time - last_mine_time >= cooldown:
                    cost = 150 + goldMine * 50
                    if gold >= cost:
                        goldMine += 1
                        gold -= cost
                        last_mine_time = current_time
                        print(f"✅ Built Goldmine #{goldMine}! (-{cost} gold)")
                    else:
                        print(f"❌ Not enough gold! You need {cost}, but have {int(gold)}.")
                else:
                    remaining = int(cooldown - (current_time - last_mine_time))
                    print(f"⏳ Goldmine is on cooldown. Wait {remaining} more seconds.")
        elif user_input == "quit":
            with lock:
                running = False
                print("💾 Saving game data...")
                break
        else:
            print("❓ Invalid command. Type 'mine' or 'quit'.")

# -----------------------------
# SAVE FUNCTION
# -----------------------------
def save_gold_history():
    global gold_history
    filename = "gold_progress.csv"
    run_id = 1

    if os.path.exists(filename):
        existing = pd.read_csv(filename)

        while f"run_{run_id}_gold" in existing.columns:
            run_id += 1

        new_data = gold_history.copy()
        new_data.rename(columns={
            "gold": f"run_{run_id}_gold",
            "goldMine": f"run_{run_id}_goldMine"
        }, inplace=True)

        merged = pd.merge(existing, new_data, on="seconds", how="outer").sort_values(by="seconds")
    else:
        gold_history.rename(columns={
            "gold": f"run_1_gold",
            "goldMine": f"run_1_goldMine"
        }, inplace=True)
        merged = gold_history

    merged.to_csv(filename, index=False)
    print(f"✅ Game data saved as Run #{run_id} to {filename}")

# -----------------------------
# MAIN THREAD HANDLING
# -----------------------------
try:
    function_thread = threading.Thread(target=gold_function)
    input_thread = threading.Thread(target=get_user_input)

    function_thread.start()
    input_thread.start()

    function_thread.join()
    input_thread.join()
except KeyboardInterrupt:
    with lock:
        running = False
        print("\n🛑 Keyboard interrupt detected. Exiting gracefully...")

# -----------------------------
# FINAL SAVE ON EXIT
# -----------------------------
with lock:
    save_gold_history()
    print("🎮 Game ended.")

In [ ]:
#3rd goldmine at 5 minutes
import time
import threading
import pandas as pd
import os

# -----------------------------
# INITIAL GAME STATE
# -----------------------------
goldMine = 1                      # Start with 1 goldmine
initial_gold = 280.0
first_mine_cost = 150
gold = initial_gold - first_mine_cost  # Subtract first goldmine cost
multiplier = 5 + (goldMine / 2)
start_time = time.time()
running = True
lock = threading.Lock()

# -----------------------------
# DATA STORAGE
# -----------------------------
gold_history = pd.DataFrame(columns=["seconds", "gold", "goldMine"])
max_duration = 2100  # 35 minutes
auto_mine_3min_added = False
auto_mine_5min_added = False

# -----------------------------
# GOLD ACCUMULATION FUNCTION
# -----------------------------
def gold_function():
    global gold, multiplier, gold_history, running
    global auto_mine_3min_added, auto_mine_5min_added, goldMine
    last_update_time = time.time()

    while running:
        time.sleep(1)
        with lock:
            current_time = time.time()
            elapsed_time = current_time - start_time

            if elapsed_time >= max_duration:
                print("\n⏰ 35 minutes have passed. Ending game.")
                running = False
                break

            # Auto-build at 3 minutes
            if not auto_mine_3min_added and elapsed_time >= 180:
                cost = 150 + goldMine * 50
                if gold >= cost:
                    gold -= cost
                    goldMine += 1
                    auto_mine_3min_added = True
                    print(f"\n⚒️ Auto-built Goldmine #{goldMine} at 3 minutes! (-{cost} gold)")
                else:
                    print(f"\n⚠️ Auto-build at 3 minutes failed: Not enough gold.")

            # Auto-build at 5 minutes
            if not auto_mine_5min_added and elapsed_time >= 300:
                cost = 150 + goldMine * 50
                if gold >= cost:
                    gold -= cost
                    goldMine += 1
                    auto_mine_5min_added = True
                    print(f"\n⚒️ Auto-built Goldmine #{goldMine} at 5 minutes! (-{cost} gold)")
                else:
                    print(f"\n⚠️ Auto-build at 5 minutes failed: Not enough gold.")

            delta_time = current_time - last_update_time
            last_update_time = current_time

            # Update multiplier and accumulate gold
            multiplier = 5 + (goldMine / 2)
            gold += delta_time * multiplier
            seconds_passed = int(elapsed_time)

            # Log values
            gold_history.loc[len(gold_history)] = [seconds_passed, int(gold), goldMine]

            # Display output
            print("\n--- Game Status ---")
            print(f"Time: {seconds_passed} sec")
            print(f"Gold: {int(gold)}")
            print(f"Multiplier: {multiplier}")
            print(f"Goldmines: {goldMine}")

# -----------------------------
# USER INPUT FUNCTION
# -----------------------------
def get_user_input():
    global gold, goldMine, running
    last_mine_time = start_time
    cooldown = 90

    while running:
        user_input = input("\nEnter 'mine' to build a Goldmine (90 sec cooldown), or 'quit' to exit: ").strip().lower()

        if user_input == "mine":
            current_time = time.time()
            with lock:
                if current_time - last_mine_time >= cooldown:
                    cost = 150 + goldMine * 50
                    if gold >= cost:
                        goldMine += 1
                        gold -= cost
                        last_mine_time = current_time
                        print(f"✅ Built Goldmine #{goldMine}! (-{cost} gold)")
                    else:
                        print(f"❌ Not enough gold! You need {cost}, but have {int(gold)}.")
                else:
                    remaining = int(cooldown - (current_time - last_mine_time))
                    print(f"⏳ Goldmine is on cooldown. Wait {remaining} more seconds.")
        elif user_input == "quit":
            with lock:
                running = False
                print("💾 Saving game data...")
                break
        else:
            print("❓ Invalid command. Type 'mine' or 'quit'.")

# -----------------------------
# SAVE FUNCTION
# -----------------------------
def save_gold_history():
    global gold_history
    filename = "gold_progress.csv"
    run_id = 1

    if os.path.exists(filename):
        existing = pd.read_csv(filename)

        while f"run_{run_id}_gold" in existing.columns:
            run_id += 1

        new_data = gold_history.copy()
        new_data.rename(columns={
            "gold": f"run_{run_id}_gold",
            "goldMine": f"run_{run_id}_goldMine"
        }, inplace=True)

        merged = pd.merge(existing, new_data, on="seconds", how="outer").sort_values(by="seconds")
    else:
        gold_history.rename(columns={
            "gold": f"run_1_gold",
            "goldMine": f"run_1_goldMine"
        }, inplace=True)
        merged = gold_history

    merged.to_csv(filename, index=False)
    print(f"✅ Game data saved as Run #{run_id} to {filename}")

# -----------------------------
# MAIN THREAD HANDLING
# -----------------------------
try:
    function_thread = threading.Thread(target=gold_function)
    input_thread = threading.Thread(target=get_user_input)

    function_thread.start()
    input_thread.start()

    function_thread.join()
    input_thread.join()
except KeyboardInterrupt:
    with lock:
        running = False
        print("\n🛑 Keyboard interrupt detected. Exiting gracefully...")

# -----------------------------
# FINAL SAVE ON EXIT
# -----------------------------
with lock:
    save_gold_history()
    print("🎮 Game ended.")


In [ ]:
#4th goldmine at 7:30
import time
import threading
import pandas as pd
import os

# -----------------------------
# INITIAL GAME STATE
# -----------------------------
goldMine = 1                      # Start with 1 goldmine
initial_gold = 280.0
first_mine_cost = 150
gold = initial_gold - first_mine_cost  # Subtract cost of first Goldmine
multiplier = 5 + (goldMine / 2)
start_time = time.time()
running = True
lock = threading.Lock()

# -----------------------------
# DATA STORAGE
# -----------------------------
gold_history = pd.DataFrame(columns=["seconds", "gold", "goldMine"])
max_duration = 2100  # 35 minutes
auto_mine_3min_added = False
auto_mine_5min_added = False
auto_mine_7min_added = False

# -----------------------------
# GOLD ACCUMULATION FUNCTION
# -----------------------------
def gold_function():
    global gold, multiplier, gold_history, running
    global auto_mine_3min_added, auto_mine_5min_added, auto_mine_7min_added, goldMine
    last_update_time = time.time()

    while running:
        time.sleep(1)
        with lock:
            current_time = time.time()
            elapsed_time = current_time - start_time

            if elapsed_time >= max_duration:
                print("\n⏰ 35 minutes have passed. Ending game.")
                running = False
                break

            # Auto-build at 3 minutes
            if not auto_mine_3min_added and elapsed_time >= 180:
                cost = 150 + goldMine * 50
                if gold >= cost:
                    gold -= cost
                    goldMine += 1
                    auto_mine_3min_added = True
                    print(f"\n⚒️ Auto-built Goldmine #{goldMine} at 3 minutes! (-{cost} gold)")
                else:
                    print(f"\n⚠️ Auto-build at 3 minutes failed: Not enough gold.")

            # Auto-build at 5 minutes
            if not auto_mine_5min_added and elapsed_time >= 300:
                cost = 150 + goldMine * 50
                if gold >= cost:
                    gold -= cost
                    goldMine += 1
                    auto_mine_5min_added = True
                    print(f"\n⚒️ Auto-built Goldmine #{goldMine} at 5 minutes! (-{cost} gold)")
                else:
                    print(f"\n⚠️ Auto-build at 5 minutes failed: Not enough gold.")

            # Auto-build at 7.5 minutes (450 seconds)
            if not auto_mine_7min_added and elapsed_time >= 450:
                cost = 150 + goldMine * 50
                if gold >= cost:
                    gold -= cost
                    goldMine += 1
                    auto_mine_7min_added = True
                    print(f"\n⚒️ Auto-built Goldmine #{goldMine} at 7.5 minutes! (-{cost} gold)")
                else:
                    print(f"\n⚠️ Auto-build at 7.5 minutes failed: Not enough gold.")

            delta_time = current_time - last_update_time
            last_update_time = current_time

            # Update multiplier and accumulate gold
            multiplier = 5 + (goldMine / 2)
            gold += delta_time * multiplier
            seconds_passed = int(elapsed_time)

            # Log values
            gold_history.loc[len(gold_history)] = [seconds_passed, int(gold), goldMine]

            # Display output
            print("\n--- Game Status ---")
            print(f"Time: {seconds_passed} sec")
            print(f"Gold: {int(gold)}")
            print(f"Multiplier: {multiplier}")
            print(f"Goldmines: {goldMine}")

# -----------------------------
# USER INPUT FUNCTION
# -----------------------------
def get_user_input():
    global gold, goldMine, running
    last_mine_time = start_time
    cooldown = 90

    while running:
        user_input = input("\nEnter 'mine' to build a Goldmine (90 sec cooldown), or 'quit' to exit: ").strip().lower()

        if user_input == "mine":
            current_time = time.time()
            with lock:
                if current_time - last_mine_time >= cooldown:
                    cost = 150 + goldMine * 50
                    if gold >= cost:
                        goldMine += 1
                        gold -= cost
                        last_mine_time = current_time
                        print(f"✅ Built Goldmine #{goldMine}! (-{cost} gold)")
                    else:
                        print(f"❌ Not enough gold! You need {cost}, but have {int(gold)}.")
                else:
                    remaining = int(cooldown - (current_time - last_mine_time))
                    print(f"⏳ Goldmine is on cooldown. Wait {remaining} more seconds.")
        elif user_input == "quit":
            with lock:
                running = False
                print("💾 Saving game data...")
                break
        else:
            print("❓ Invalid command. Type 'mine' or 'quit'.")

# -----------------------------
# SAVE FUNCTION
# -----------------------------
def save_gold_history():
    global gold_history
    filename = "gold_progress.csv"
    run_id = 1

    if os.path.exists(filename):
        existing = pd.read_csv(filename)

        while f"run_{run_id}_gold" in existing.columns:
            run_id += 1

        new_data = gold_history.copy()
        new_data.rename(columns={
            "gold": f"run_{run_id}_gold",
            "goldMine": f"run_{run_id}_goldMine"
        }, inplace=True)

        merged = pd.merge(existing, new_data, on="seconds", how="outer").sort_values(by="seconds")
    else:
        gold_history.rename(columns={
            "gold": f"run_1_gold",
            "goldMine": f"run_1_goldMine"
        }, inplace=True)
        merged = gold_history

    merged.to_csv(filename, index=False)
    print(f"✅ Game data saved as Run #{run_id} to {filename}")

# -----------------------------
# MAIN THREAD HANDLING
# -----------------------------
try:
    function_thread = threading.Thread(target=gold_function)
    input_thread = threading.Thread(target=get_user_input)

    function_thread.start()
    input_thread.start()

    function_thread.join()
    input_thread.join()
except KeyboardInterrupt:
    with lock:
        running = False
        print("\n🛑 Keyboard interrupt detected. Exiting gracefully...")

# -----------------------------
# FINAL SAVE ON EXIT
# -----------------------------
with lock:
    save_gold_history()
    print("🎮 Game ended.")
